# Apprentissage Automatique : BE 2 

## Premier apprentissage

In [4]:
import numpy as np
import random
# from mnist import MNIST

In [ ]:
np.random.seed(1) # pour que l'exécution soit déterministe

# N est le nombre de données d'entrée
# D_in est la dimension des données d'entrée
# D_h le nombre de neurones de la couche cachée
# D_out est la dimension de sortie (nombre de neurones de la couche de sortie

N, D_in, D_h, D_out = 30, 2, 10, 3
lr = 0.01 # learning rate


In [9]:
def forwardPass(X, W, b):
    I = X.dot(W)+b
    O = 1/(1+np.exp(-I))
    return (O)

In [7]:
def backwardPass(X, Y, e, W, b, lr):
    new_W = W + (lr*Y*(1-Y)*e) * X.T
    new_b = b + (lr*Y*(1-Y)*e)
    new_e = e*W
    return new_W, new_b, new_e

In [43]:
def printLoss(ii,lLoss):
    if not ii%25000:
        lLoss.append(np.square(Y_pred -Y).sum() / 2)
    return lLoss

In [18]:
def sigmoidDiff(Y):
    return Y*(1-Y)

In [22]:
def backProgLastLayer(Y,Y_pred):
    error=Y-Y_pred
    delta=error*sigmoidDiff(Y_pred)
    return error,delta

In [27]:
def backProgHiddenLayer(sup_delta,sup_W,O):
    error=sup_delta.dot(sup_W.T)
    delta=error*sigmoidDiff(O1)
    return error,delta

In [46]:
def updateFinalLayer(delta,lr,I,W,b,O):
    W += lr * 

SyntaxError: unexpected EOF while parsing (<ipython-input-46-71177c283b02>, line 1)

In [45]:
np.random.seed(1)
lLoss=[]

X = np.random.random((N, D_in))
Y = np.random.random((N, D_out))


'''Initialisation aléatoire des poids du réseau'''
W1 = 2 * np.random.random((D_in, D_h)) - 1
b1 = np.zeros((1, D_h))
W2 = 2 * np.random.random((D_h, D_out)) - 1
b2 = np.zeros((1, D_out))

for ii in range (100000):
    '''Forward '''

    O1 = forwardPass(X,W1,b1)
    O2 = forwardPass(O1,W2,b2)
    Y_pred = O2

    lLoss=printLoss(ii,lLoss)

    ''' Backprog ''' 
    l2_error,l2_delta=backProgLastLayer(Y,Y_pred)
    l1_error,l1_delta=backProgHiddenLayer(l2_delta,W2,O1)

    W2 += lr * O1.T.dot(l2_delta)
    b2 += lr * l2_delta.sum(axis=0)
    O1 += lr * l2_delta.dot(W2.T)

    b1 += lr * l1_delta.sum(axis=0)
    W1 += lr * X.T.dot(l1_delta)
print(lLoss)

[4.668071349518289, 3.4877867235805384, 3.176335850400406, 2.421024553266937]
